# Código para aplicar o melhor resultado obtido no ajuste da MLP

In [9]:
import os
import GPUtil
import psutil
import rasterio
import numpy as np
import pandas as pd
import xarray as xr
import asciichartpy
import geopandas as gpd
import rioxarray as rxr

from tqdm import tqdm, trange
from pyproj import Transformer
from utils.consts import SOIL_TYPES
from rasterio.transform import rowcol
from IPython.display import clear_output
from concurrent.futures import ThreadPoolExecutor, as_completed 

import torch
import torch.nn as nn

from Model1.codes.mlp import MLP
from Model1.codes.dataset import MeuDataset

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


### Carregando os dados salvos

In [11]:
save = torch.load(r"Model1\143_6340058853885.pth")

In [12]:
dataset = MeuDataset(device=device, eval=True)

Lendo Tabelas
Lendo Rasteres
Processando dados


In [13]:
mlp = MLP().to(device=device)
mlp.load_state_dict(save['model_state'])
mlp.eval()  # entra em modo de inferência

mlp(*dataset[0])

tensor([[nan]], device='cuda:0', dtype=torch.float64, grad_fn=<AddmmBackward0>)

In [ ]:
step = 50
values = np.zeros(len(dataset), dtype=np.float32)

def tarefa(idx):
    x = dataset[idx]
    y_pred = mlp(*x)

    return idx, y_pred[:, 0]

with torch.no_grad():  # evita cálculo de gradiente
    futuros = []
    for idx in trange(0, len(dataset), step, desc="Processando"):
        start = idx
        end = min(len(dataset), start+step)

        idx, val = tarefa(slice(start, end))
        values[idx] = [i.item() for i in val]

values = np.reshape(values, dataset.uso_solo.values[0].shape) # type: ignore
values

In [ ]:
coords = {"y": dataset.uso_solo.y, "x": dataset.uso_solo.x} # type: ignore

da = xr.DataArray(
    values,
    coords=coords,
    dims=("y","x"),
    name="K"
)
da = da.expand_dims("band").assign_coords(band=[1])

stack = da.rio.write_crs(dataset.uso_solo.rio.crs) # type: ignore
stack = stack.rio.write_transform(dataset.uso_solo.rio.transform()) # type: ignore
stack.rio.to_raster(fr"D:/Mestrado/Trabalho Final/SIG/K_espacializado.tif", compress="LZW")

In [ ]:
values[~np.isnan(values)].min()

np.float64(9.135550702056427)